# alpha

In [1]:
import pandas as pd

In [2]:
from zipline.research import prices

In [3]:
# 尽量选择同时包含正负收益的股票列表
ps = prices(['000001','000002','000333','600645','600771'],'2018-5-19','2018-5-29')

[2018-06-01 15:04:05.503512] INFO: zipline.finance.metrics.tracker: 模拟1个交易日
首个开盘时间: 2018-05-29 01:31:00+00:00
最后收盘时间: 2018-05-29 07:00:00+00:00


In [4]:
alphas = ps.pct_change(6).iloc[-1]

## 修改数据以利更好观察

In [5]:
alphas = pd.Series([-0.1,-0.05,-0.02,0,0.07],index=alphas.index)

In [6]:
alphas

平安银行(000001)   -0.10
万 科Ａ(000002)   -0.05
美的集团(000333)   -0.02
中源协和(600645)    0.00
广誉远(600771)     0.07
dtype: float64

In [7]:
_long, _short = alphas.index[-1], alphas.index[0]

# 优化

## 期初投资组合

In [8]:
current_portfolio = pd.Series([0.2] * 5, index=alphas.index)

In [9]:
current_portfolio

平安银行(000001)    0.2
万 科Ａ(000002)    0.2
美的集团(000333)    0.2
中源协和(600645)    0.2
广誉远(600771)     0.2
dtype: float64

In [10]:
universe = current_portfolio.index

In [11]:
universe.get_indexer([_long,_short])

array([4, 0])

## 最大alpha

In [12]:
from zipline import optimize as opt

In [13]:
objective = opt.MaximizeAlpha(alphas)

In [19]:
constraints = [
    opt.MaxGrossExposure(3),
    #opt.Frozen(current_portfolio.index[2:4]),
    #opt.NotLessThan(0.1),
    opt.NotExceed(1.0),
    opt.Pair(_long, _short, 2),
]

In [20]:
result = opt.run_optimization(objective, constraints, current_portfolio)

In [21]:
result.print_diagnostics()

求解状态              成功
最优解         0.180000
迭代次数              10
设立问题用时(s)   0.000060
求解问题用时(s)   0.000112
求解器名称           ECOS


In [22]:
result.new_weights

,long,short,total
平安银行(000001),0.0,-0.5,0.5
万 科Ａ(000002),0.0,-1.0,1.0
美的集团(000333),0.0,-0.5,0.5
中源协和(600645),0.0,-0.0,0.0
广誉远(600771),1.0,-0.0,1.0


In [23]:
opt.calculate_optimal_portfolio(objective, constraints, current_portfolio)

,long,short,total
平安银行(000001),0.0,-0.5,0.5
万 科Ａ(000002),0.0,-1.0,1.0
美的集团(000333),0.0,-0.5,0.5
中源协和(600645),0.0,-0.0,0.0
广誉远(600771),1.0,-0.0,1.0
